# Test Insert Operations

## Setup

In [ ]:
from fastlite import *

Note: Make sure to use fastlite's `database()` here

In [ ]:
%%aip 0
Create simple tests for kw.insert() that change nothing

In [ ]:
db = database(':memory:')

In [ ]:
t = db.t.test

In [ ]:
class People: id: int; name: str

In [ ]:
people = db.create(People, pk='id')

## Test Single Inserts

Here we test `insert()`

### Test Cases for `insert()` Where Nothing Is Inserted

Test that calling `insert()` without any parameters doesn't change anything, and returns nothing

In [ ]:
people.insert()

ZeroDivisionError: integer division or modulo by zero

Test None doesn't change anything. Note: The code below raises ZeroDivisionError: integer division or modulo by zero

In [ ]:
# count = people.count
# try: people.insert(None)
# except (TypeError, AttributeError): pass  # Expected failure
# assert people.count == count

Test empty dict doesn't change anything 

In [ ]:
count = people.count
try: people.insert({})
except (TypeError, AttributeError): pass  # Expected failure
assert people.count == count

ZeroDivisionError: integer division or modulo by zero

In [ ]:
# Test empty dataclass doesn't change anything
PersonDC = people.dataclass()
count = people.count
try: people.insert(PersonDC())
except (TypeError, AttributeError): pass  # Expected failure
assert people.count == count

# Test empty class instance doesn't change anything
class EmptyPerson: pass
count = people.count
try: people.insert(EmptyPerson())
except (TypeError, AttributeError): pass  # Expected failure
assert people.count == count

### Single Insert Types

Test insert with keyword argument. Result should be the inserted item.

In [ ]:
result = people.insert(name='Alice')
print(result)
assert result.name == 'Alice'

People(id=1, name='Alice')


Test insert with dataclass

In [ ]:
result = people.insert(People(name='Bobba'))
print(result)
assert result.name == 'Bobba'

People(id=2, name='Bobba')


Test with regular class

In [ ]:
class Student: pass
student = Student()
student.name = 'Charlo'

result = people.insert(student)
print(result)
assert result.name == 'Charlo'

People(id=3, name='Charlo')


Verify count is 3

In [ ]:
assert people.count == 3

### None and Empty String Handling

SQLite makes a clear distinction between NULL (represented as None in Python) and an empty string (''). Unlike some popular Python ORMs, fastlite preserves this distinction because:

1. NULL represents "unknown" or "missing" data
2. Empty string represents "known to be empty"

These are semantically different concepts, and maintaining this distinction allows users to make appropriate queries (e.g. `WHERE name IS NULL` vs `WHERE name = ''`). The fact that fastlite preserves this distinction in both directions (Python->SQLite and SQLite->Python) is good database design.

Test inserting a record with name set to None

In [ ]:
result = people.insert(name=None)
print(result)
assert result.name is None

People(id=4, name=None)


Test with empty string

In [ ]:
result = people.insert(name='')
print(result)
assert result.name == ''

People(id=5, name='')


In [ ]:
people.get(pk_values=4)

People(id=4, name=None)

In [ ]:
people.get(pk_values=5)

People(id=5, name='')

In [ ]:
print(db.q(f'SELECT * FROM people WHERE id IN (4,5)'))

[{'id': 4, 'name': None}, {'id': 5, 'name': ''}]


Remember, `get()` is for getting single items. The following would not work here. `pk_values` can be a list only for tables with compound primary keys.

```python
# people.get(pk_values=[4,5])
```

### Other Cases

Test with special characters

In [ ]:
assert people.insert(name='O\'Connor').name == "O'Connor"
assert people.insert(name='José').name == 'José'

Test id auto-increment

In [ ]:
p1 = people.insert(name='Test1')
p2 = people.insert(name='Test2') 
assert p2.id == p1.id + 1

Test dict insert

In [ ]:
assert people.insert({'name': 'Dict Test'}).name == 'Dict Test'

Test that extra fields raise `sqlite3.OperationalError`

In [ ]:
from sqlite3 import OperationalError

In [ ]:
try:
    p = people.insert(name='Extra', age=25, title='Dr')
except OperationalError as e:
    assert e.args[0] == 'table people has no column named age'

## Test Multiple Inserts

Here we test `insert_all()`

### Test cases for `insert_all()` where nothing is changed

Test empty list doesn't change anything

In [ ]:
count = people.count
people.insert_all([])
assert people.count == count

Test other empty iterables don't change anything

In [ ]:
count = people.count
people.insert_all(iter([]))  # empty iterator
people.insert_all(set())     # empty set
people.insert_all(tuple())   # empty tuple
assert people.count == count

Test that lists of `None` don't change anything - note: the next 2 cases fail due to [flexiclass](https://fastcore.fast.ai/xtras.html#flexiclass) behavior

In [ ]:
# count = people.count
# people.insert_all([None, None])
# assert people.count == count

In [ ]:
# count = people.count
# people.insert_all([None])
# assert people.count == count

### Test cases for `insert_all()` where records are inserted

Test list of dicts

In [ ]:
count = people.count
data = [{'name': 'Bulk1'}, {'name': 'Bulk2'}, {'name': 'Bulk3'}]
people.insert_all(data)
assert people.count == len(data) + count

In [ ]:
people()

[People(id=1, name='Alice'),
 People(id=2, name='Bobba'),
 People(id=3, name='Charlo'),
 People(id=4, name=None),
 People(id=5, name=''),
 People(id=6, name="O'Connor"),
 People(id=7, name='José'),
 People(id=8, name='Test1'),
 People(id=9, name='Test2'),
 People(id=10, name='Dict Test'),
 People(id=11, name='Bulk1'),
 People(id=12, name='Bulk2'),
 People(id=13, name='Bulk3')]

Test `insert_all` with a list of dataclass instances to insert

In [ ]:
count = people.count
Person = people.dataclass()
data = [Person(name=f'DC{i}') for i in range(3)]
result = people.insert_all(data)
print(result)
assert people.count == count + 3

"people"


Test list of regular class instances

In [ ]:
count = people.count
class Student:
    def __init__(self, name): self.name = name
students = [Student(f'Student{i}') for i in range(3)]
people.insert_all(students)
assert people.count == count + 3

### Edge Cases

Test mixed types in list

In [ ]:
count = people.count
Person = people.dataclass()
mixed_data = [
    {'name': 'Dict1'},
    Person(name='DC1'),
    Student('Student1')
]
people.insert_all(mixed_data)
assert people.count == count + 3

Test None/empty strings in bulk insert

In [ ]:
count = people.count
null_data = [
    {'name': None},
    {'name': ''},
    {'name': 'Regular'}
]
people.insert_all(null_data)
assert people.count == count + 3

Test with special characters in bulk

In [ ]:
count = people.count
special_data = [
    {'name': "O'Brien"},
    {'name': 'José'},
    {'name': '张伟'}
]
people.insert_all(special_data)
assert people.count == count + 3

Test error on invalid column

In [ ]:
try:
    people.insert_all([{'name': 'Valid'}, {'invalid_col': 'Bad'}])
except OperationalError as e:
    assert 'no column named invalid_col' in str(e)